In [3]:
!pip3 install pandas
!pip3 install scikit-learn
from typing import List
from math import log
import numpy as np
import pandas as pd
import random
from sklearn.metrics.pairwise import cosine_similarity
# for printing out all the columns of a pandas dataframe https://towardsdatascience.com/how-to-show-all-columns-rows-of-a-pandas-dataframe-c49d4507fcf
pd.set_option('display.max_columns', None)

In [4]:
CORPUS = [
    'this is the first document',
    'the first document is this',
    'this is the second document',
    'and this is the third document',
    'is this the first document'
]

In [6]:
# 문서(doc)안에서 단어의 빈도수를 구하는 함수
# 문서와 단어를 인자로 받아온다
# 문서를 띄어쓰기로 구분한 후, 받아온 단어와 비교한다. 같으면 T, 다르면 F
# 각 논리 값의 sum을 구해준다
def tf(term: str, doc: str) -> int:
    tf = sum([
      1 if word == term else 0      
      for word in doc.split(" ")
    ])
    return tf


# tf를 matrix 형태로 표현하는 함수
# 말뭉치(corpus)를 인자로 받아온다
# corpus 안의 문서를 띄어쓰기로 구분하여 단어를 가져온다 (중복 O)
# set으로 묶어 중복을 없애주고, 이를 다시 list로 묶어준다
def build_dtm(corpus: List[str]) -> pd.DataFrame:
    words = [
        word
        for doc in corpus
        for word in doc.split(" ")
    ]
    vocab = list(set(words))


    # 중복없는 단어들(vocab), corpus 안의 문서(doc)을 tf의 인자로 전달한다
    # tf가 단어의 빈도수를 반환하여 dtm에 들어간다
    dtm = [
           [tf(term, doc) for term in vocab]
           for doc in corpus
    ]

    # 데이터프레임으로 바꿔준다
    dtm = pd.DataFrame(data=dtm, columns=vocab)
    return dtm

In [7]:
# 단어가 등장하는 문장의 수, 그것의 역수를 구하는 함수
# 문서를 띄어쓰기로 구분한 후, 받아온 단어들 속에 있으면 T, 없으면 F
# 각 논리 값의 sum을 구한다
# idf 공식 그대로 구현해준다
def idf(term: str, corpus: List[str]) -> float:
    n = len(corpus)
    df = sum([
              term in doc.split(" ")
              for doc in corpus
    ])
    idf = log(n / (1 + df))
    return idf

In [8]:
# tfidf 구현하는 함수 (tf * idf)
# tf = dtm → build_dtm(corpus)
# idf = idfs → dtm의 컬럼의 단어들과 corpus를 idf의 인자로 전달
def build_dtm_with_tfidf(corpus: List[str]) -> pd.DataFrame:
    dtm = build_dtm(corpus)

    idfs: List[float] = [
                         idf(term, corpus)
                         for term in dtm.columns
    ]
    dtm_tfidf: List[float] = dtm.to_numpy() * np.array(idfs)
     
    return pd.DataFrame(data=dtm_tfidf, columns=dtm.columns)

In [9]:
dtm = build_dtm(CORPUS)
dtm_tfidf = build_dtm_with_tfidf(CORPUS)
print(dtm)
print(dtm_tfidf)

   third  first  the  and  this  is  second  document
0      0      1    1    0     1   1       0         1
1      0      1    1    0     1   1       0         1
2      0      0    1    0     1   1       1         1
3      1      0    1    1     1   1       0         1
4      0      1    1    0     1   1       0         1
      third     first       the       and      this        is    second  \
0  0.000000  0.223144 -0.182322  0.000000 -0.182322 -0.182322  0.000000   
1  0.000000  0.223144 -0.182322  0.000000 -0.182322 -0.182322  0.000000   
2  0.000000  0.000000 -0.182322  0.000000 -0.182322 -0.182322  0.916291   
3  0.916291  0.000000 -0.182322  0.916291 -0.182322 -0.182322  0.000000   
4  0.000000  0.223144 -0.182322  0.000000 -0.182322 -0.182322  0.000000   

   document  
0 -0.182322  
1 -0.182322  
2 -0.182322  
3 -0.182322  
4 -0.182322  


다음과 같은 결과가 나와야 합니다 (단어의 순서는 달라도 괜찮습니다):
```
   the  and  third  this  is  first  document  second
0    1    0      0     1   2      1         1       0
1    1    0      0     1   2      1         1       0
2    1    0      0     1   2      0         1       1
3    1    1      1     1   2      0         1       0
4    1    0      0     1   2      1         1       0
        the       and     third      this        is     first  document  \
0 -0.182322  0.000000  0.000000 -0.182322 -0.364643  0.223144 -0.182322   
1 -0.182322  0.000000  0.000000 -0.182322 -0.364643  0.223144 -0.182322   
2 -0.182322  0.000000  0.000000 -0.182322 -0.364643  0.000000 -0.182322   
3 -0.182322  0.916291  0.916291 -0.182322 -0.364643  0.000000 -0.182322   
4 -0.182322  0.000000  0.000000 -0.182322 -0.364643  0.223144 -0.182322   

     second  
0  0.000000  
1  0.000000  
2  0.916291  
3  0.000000  
4  0.000000  

```

In [13]:
# 코사인 유사도를 비교
print(cosine_similarity(dtm.to_numpy(), dtm.to_numpy()))
print("\n")
print(cosine_similarity(dtm_tfidf.to_numpy(), dtm_tfidf.to_numpy()))

[[1.         1.         0.8        0.73029674 1.        ]
 [1.         1.         0.8        0.73029674 1.        ]
 [0.8        0.8        1.         0.73029674 0.8       ]
 [0.73029674 0.73029674 0.73029674 1.         0.73029674]
 [1.         1.         0.8        0.73029674 1.        ]]


[[1.         1.         0.31538537 0.23104796 1.        ]
 [1.         1.         0.31538537 0.23104796 1.        ]
 [0.31538537 0.31538537 1.         0.10015744 0.31538537]
 [0.23104796 0.23104796 0.10015744 1.         0.23104796]
 [1.         1.         0.31538537 0.23104796 1.        ]]


다음과 같은 결과가 나와야 합니다:
```
[[1.         1.         0.875      0.82495791 1.        ]
 [1.         1.         0.875      0.82495791 1.        ]
 [0.875      0.875      1.         0.82495791 0.875     ]
 [0.82495791 0.82495791 0.82495791 1.         0.82495791]
 [1.         1.         0.875      0.82495791 1.        ]]
[[1.         1.         0.4227912  0.31662902 1.        ]
 [1.         1.         0.4227912  0.31662902 1.        ]
 [0.4227912  0.4227912  1.         0.16251445 0.4227912 ]
 [0.31662902 0.31662902 0.16251445 1.         0.31662902]
 [1.         1.         0.4227912  0.31662902 1.        ]]

```

## 다음의 질문에 답해주세요!

> `dtm`으로 구한 유사도 행렬 대비, `dtm_tfidf`로 구한 유사도 행렬은 어떤 점이 개선되었나요? 그 이유는?

개선된 점:
유사도 행렬이 좀 더 정확해졌다. 다른 의미인 문장들의 유사도는 내려간다.
- 홍예은09:16
유사도 행렬의 값들이 다양해집니다
- 박민우09:17
값 사이의 차이가 커져서 분리하기 쉽다
 
이유:
-  홍예은09:14
단순 개수가 아닌 상대적 빈도를 계산하기 때문에 불용어의 중요도가 낮아집니다. 문서를 대변할만한 단어의 중요도가 올라간다.
- 박민우09:15
불용어가 문장의 keyword가 되는 것을 방지합니다
- 박재운09:16
각 단어의 중요한 값의 가중치를 부여하여 중요한 단어를 잘 표현할 수 있음

IDF(t) = log(n) / 1 + DF(t) 수식 에서 (n) 값이랑 DF(t) 값이 차이가 1이면 분자랑 분모가 같아져서 값이 0이 나옵니다.


> `dtm_tfidf`로도 해결할수 없는 문제를 발견할 수 있나요?

- 홍예은09:19
1. 여전히 어순을 인지하지 못한다. ('this is the first document' == 'is this the first document')
2. 드물게 나타나지만 문서를 구성하는 요소 또한 키워드로 인식한다. (e.g. 'and', 'of' ...)
3. 단어의 변형을 인지하지 못하므로 (e.g. document, documents) 전처리에 영향을 크게 받는다. 
4. 키워드 판별 척도가 빈도이므로 동음이의어, 다의어, 단어의 개념적 유사도(e.g. 한국, 대한민국)를 고려하지 못한다.
5. 중요한 단어도 말뭉치 안에서 자주 사용될 수 있다. (전제의 오류)


- 박민우09:20
문장 어순을 인지할 수 없음. **합성어와 그를 이루는 단어가 함께 나올 경우**,  keyword를 잘못 결정하는 상황이 발생할 수 있음

e.g. -> after rain, The rain + bow appeared.
subword tokenization (필요한 이유)

